In [40]:
#Python program to scrape website
#and save quotes from website
import requests
from bs4 import BeautifulSoup
import csv
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time

browser = webdriver.Chrome('/Users/Xcalibre/chrome_driver/chromedriver')

In [41]:
import numpy as np
import pandas as pd

In [42]:
def login():
    email = 'teq00525@zwoho.com'
    passwd = 'qwerty'
    browser.get('https://www.screener.in/login/')
    browser.find_element_by_xpath('//*[@id="id_username"]').send_keys(email)
    time.sleep(2)
    browser.find_element_by_xpath('//*[@id="id_password"]').send_keys(passwd)
    time.sleep(2)
    browser.find_element_by_xpath('/html/body/main/div/div/div[2]/form/button').click()

In [43]:
# returns the number of categories 'domains'.
def number_of_categories():
    URL = "https://www.screener.in/explore/"
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html5lib')
    labels = {}
    table = soup.find('div', attrs = {'class':'flex-row flex-gap-12'})
    categories = table.findAll('a', attrs = {'class':'bordered radius-6 padding-4-12 font-size-14 ink-700'})
    for c in categories:
        labels[c.text] = 'https://www.screener.in/user/columns/?next=' + c['href']
    return labels

In [44]:
def tag_list():
    required_tags = ['OPM', 'YOY Quarterly sales growth', 'YOY Quarterly profit growth', 
                     'Return on capital employed', 'Return on equity']

    browser.find_element_by_xpath('//*[@id="manage-reset"]').click()
    time.sleep(3)
    page = browser.page_source
    soup = BeautifulSoup(page, 'html5lib')
    table = soup.find('ul', attrs = {'class':'items flex-row'})
    categories = table.findAll('li', attrs = {'class':'draggable-row'})
    current_tags = [c['data-name'] for c in categories]

    tags_to_add = list(set(required_tags) - set(current_tags))
    tags_to_remove = list(set(current_tags) - set(required_tags))
    process_tags = tags_to_add + tags_to_remove
    process_tags.remove('Net Profit latest quarter')
    return process_tags

In [45]:
def choose_tags(process_tags):
#     browser.find_element_by_xpath('//*[@id="manage-menu"]/li[4]/button').click()
    for tag in process_tags[:2]:
        time.sleep(1)
        xpath = f'//*[@value="{tag}"]'
#         print(xpath)
        browser.find_element_by_xpath(xpath).click()
    time.sleep(1)
    browser.refresh()
    time.sleep(3)
    clk_btn = browser.find_element_by_xpath('/html/body/main/div/div[2]/form/div[1]/div/button')
    browser.execute_script("arguments[0].click();", clk_btn)

In [46]:
def crawl_table(url):
#     url = 'https://www.screener.in/company/compare/00000002/'
    browser.get(url)
    page = browser.page_source
    soup = BeautifulSoup(page, 'html5lib')
    table = soup.find('table', attrs = {'class':'data-table text-nowrap striped'})
    rows = table.findAll('tr')
    col_names = [c.text.strip().split('\n')[0] for c in rows[0].findAll('th')]
    data = []
    for r in rows[1:]:
        row_data = [d.text.strip() for d in r.findAll('td')]
        data.append(row_data)

    df = pd.DataFrame(data, columns=col_names)
    df.set_index('S.No.')

    return df

In [47]:
login()
domains = number_of_categories()

In [48]:
# for i in tqdm(range(len(domains))):
for i in tqdm(range(4,6)):
    domain_name = list(domains.keys())[i].replace('/', '_or_')
    url = list(domains.values())[i]
    try:
        browser.get(url)
        process_tags = tag_list()
        print(process_tags)
        choose_tags(process_tags)
        time.sleep(2)
        page = browser.page_source
        soup = BeautifulSoup(page, 'html5lib')
        total_pages = int(soup.find('div', attrs = {'class':'flex-row flex-gap-8 flex-space-between flex-align-center'}).find('div', 
                    attrs = {'class':'sub'}).text.replace('\n','').strip().split(' ')[-1])
        dataframes = []
        for p in range(total_pages):
            current_url = browser.current_url + f'?limit=50&page={p+1}'
            dataframes.append(crawl_table(current_url))
        dataframes = pd.concat(dataframes).dropna()
        dataframes.to_csv(f'{domain_name}.csv', index=False)
    except:
        print(f'{domain_name} -- {url}')

  0%|          | 0/2 [00:00<?, ?it/s]

['Return on equity', 'OPM', 'Price to Earning', 'Market Capitalization', 'Dividend yield', 'Sales latest quarter']


 50%|█████     | 1/2 [00:15<00:15, 15.25s/it]

['Return on equity', 'OPM', 'Price to Earning', 'Market Capitalization', 'Dividend yield', 'Sales latest quarter']


100%|██████████| 2/2 [00:30<00:00, 15.04s/it]


In [49]:
# df = pd.read_csv('Chemicals.csv')
dataframes

,S.No.,Name,CMP,NP Qtr,ROCE,Mar Cap
0,1.,HDFC Bank,1486.35,8433.78,6.76,820174.54
1,2.,ICICI Bank,635.70,4886.16,6.04,440128.10
2,3.,St Bk of India,429.60,6125.52,4.53,383401.31
3,4.,Kotak Mah. Bank,1794.35,2589.32,6.65,355709.79
4,5.,Axis Bank,738.45,2941.41,5.06,226336.85
5,6.,IndusInd Bank,1015.10,926.22,6.54,78547.94
6,7.,Bandhan Bank,319.40,103.03,8.92,51443.03
7,8.,Punjab Natl.Bank,42.05,700.80,6.52,46301.32
8,9.,Bank of Baroda,84.45,-740.79,4.33,43672.15
9,10.,IDBI Bank,39.00,629.05,4.73,41934.37


In [22]:
domains

{'Agro Chemicals': 'https://www.screener.in/user/columns/?next=/company/compare/00000001/',
 'Air Transport Service': 'https://www.screener.in/user/columns/?next=/company/compare/00000002/',
 'Alcoholic Beverages': 'https://www.screener.in/user/columns/?next=/company/compare/00000003/',
 'Auto Ancillaries': 'https://www.screener.in/user/columns/?next=/company/compare/00000004/',
 'Automobile': 'https://www.screener.in/user/columns/?next=/company/compare/00000005/',
 'Banks': 'https://www.screener.in/user/columns/?next=/company/compare/00000006/',
 'Bearings': 'https://www.screener.in/user/columns/?next=/company/compare/00000066/',
 'Cables': 'https://www.screener.in/user/columns/?next=/company/compare/00000007/',
 'Capital Goods - Electrical Equipment': 'https://www.screener.in/user/columns/?next=/company/compare/00000008/',
 'Capital Goods-Non Electrical Equipment': 'https://www.screener.in/user/columns/?next=/company/compare/00000009/',
 'Castings, Forgings & Fastners': 'https://www.

In [20]:
df = pd.read_csv('Banks.csv')

In [21]:
df

,S.No.,Name,CMP,NP Qtr,Qtr Profit Var,Qtr Sales Var,Mar Cap
0,1.0,HDFC Bank,1486.35,8433.78,15.85,2.55,820174.54
1,2.0,ICICI Bank,635.70,4886.16,290.48,2.57,440128.10
2,3.0,St Bk of India,429.60,6125.52,19.39,3.27,383401.31
3,4.0,Kotak Mah. Bank,1794.35,2589.32,-21.60,-5.72,355709.79
4,5.0,Axis Bank,738.45,2941.41,333.00,-4.02,226336.85
5,6.0,IndusInd Bank,1015.10,926.22,193.80,0.44,78547.94
6,7.0,Bandhan Bank,319.40,103.03,-80.08,5.42,51443.03
7,8.0,Punjab Natl.Bank,42.05,700.80,185.41,34.07,46301.32
8,9.0,Bank of Baroda,84.45,-740.79,-261.24,-8.88,43672.15
9,10.0,IDBI Bank,39.00,629.05,284.50,3.25,41934.37
